In [1]:
import pandas as pd
import keras
from keras import layers
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import mean_squared_error
import tensorflow as tf
import matplotlib

## Add this line for reproducibility reasons (to have the same random seed in all keras layers)
tf.keras.utils.set_random_seed(100)

In [2]:
def error(dfin, dfout):
    """
    Calculate the error between the predicted and actual
    """
    t_error = (dfin[:, 0] - dfout[:, 0]) ** 2
    p_error = (dfin[:, 1] - dfout[:, 1]) ** 2
    d_error= (dfin[:, 2] - dfout[:, 2]) ** 2
    error_all = t_error + p_error + d_error
    error_df = pd.DataFrame([t_error, p_error, d_error, error_all]).transpose()
    
    return error_df

In [3]:
def plot_sensitivity(_df_results, name, _save_params):
    """
    Creates a 3D plot with all Diffusivity values over temperature and pressure. The experimental results are also displayed in this plot. 
    
    """    
    _md = pd.read_csv('../data/CO2_clean.csv', usecols=['Index', 'T', 'P', 'D'])
    
    
    
    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(projection='3d')
    
    ax.scatter(_df_results['T'], _df_results['P'], _df_results['D'],alpha=0.2)
    ax.scatter(_md['T'], _md['P'], _md['D'],alpha=0.5, s=80, marker='^')
    ax.legend(['Predictions', 'MD'], loc='upper left')
    
    ax.set_xlabel('Temperature\n(K)')
    ax.set_ylabel('Pressure\n(MPa)')
    ax.set_zlabel('Diffusivity\n($10^{-9} m^2/s$)')
    
    if _save_params['save_figure']:
        plt.savefig(_save_params['save_path'] + '/{}_sensitivity.png'.format(name))
    plt.show()

In [5]:
def export_results_autoencoder(_model, _scaler):
    """
    Exports the trained autoencoder 
    """

    diffusivities = np.arange(0, 25, 25 / 1000)  # create a vector with small step to locate the minimum error

    _results_df = pd.DataFrame()
    for i in range(275, 625, 10):
        temperatures = [i for d in range(len(diffusivities))]
        for j in range(1, 1000, 10):
            Pressure = j / 10

            pressures = [Pressure for i in range(len(temperatures))]
            temp_max = 625
            # temp_max = 369.9 * Pressure ** (0.089)
            temp = {'T': temperatures, 'P': pressures, 'D': diffusivities}
            _df_for_test = pd.DataFrame(temp)
            _df_for_test = _df_for_test[_df_for_test['T'] < temp_max]
            try:
                _temp_df_scaled = _scaler.transform(_df_for_test)
                _results = _model.predict(_temp_df_scaled)
                # use the error function to get the error values for all T and P pairs
                _error_df = error(_temp_df_scaled, _results)
                # find the diffusivity value for the specific P and T with minimum error
                _min_error = _error_df[[3]].idxmin()
                _best_value = _df_for_test.iloc[_min_error]
                _results_df = pd.concat([_results_df, _best_value], ignore_index=True)


            except:
                continue



    return _results_df

In [6]:

if __name__ == "__main__":
    
    %matplotlib notebook
    save_params ={'path': '../results/',
                  'save_figure' : False}
    
    # load the data
    df = pd.read_csv('../data/CO2_clean.csv', usecols=['Index', 'T', 'P', 'D'])
    X = df[['T', 'P', 'D']]
    y = df['D']

    ###########################
    ####### AUTOENCODER #######
    ###########################
  
    ## Set callbacks ##
    early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=15, verbose=1, mode='auto')
    reduce_lron_plateau = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=1, mode='auto')
    
    ##  Define the Architecture ##
    input_layer = keras.Input(shape=(3,), name='input_layer')
    encoder_layer_1 = layers.Dense(44, activation='elu', name='encoder_layer_1')(input_layer)
    encoder_layer_2 = layers.Dense(72, activation='elu', name='encoder_layer_2')(encoder_layer_1)
    latent = layers.Dense(2, activation='elu', name='latent_space')(encoder_layer_2)
    decoder_layer_1 = layers.Dense(72, activation='elu', name='decoder_layer_1')(latent)
    decoder_layer_2 = layers.Dense(44, activation='elu', name='decoder_layer_2')(decoder_layer_1)
    output_layer = layers.Dense(3, activation='elu', name='output_layer')(decoder_layer_2)
    
    scaler_full = MinMaxScaler()
    X_scaled = scaler_full.fit_transform(X)
    autoencoder_full = keras.Model(input_layer, output_layer)
    autoencoder_full.compile(optimizer='adam', loss='mse')
    history_full = autoencoder_full.fit(X_scaled, X_scaled,
                                   verbose=0,
                                   epochs=1000,
                                   batch_size=32,
                                   shuffle=False,
                                   validation_data=(X_scaled, X_scaled),
                                   callbacks=[reduce_lron_plateau, early_stopping])
    
    df_auto_results_full = export_results_autoencoder(autoencoder_full, scaler_full)
    plot_sensitivity(df_auto_results_full,'Autoencoder',save_params)


Epoch 28: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 47: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.

Epoch 62: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.

Epoch 72: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.

Epoch 82: ReduceLROnPlateau reducing learning rate to 3.125000148429535e-05.

Epoch 92: ReduceLROnPlateau reducing learning rate to 1.5625000742147677e-05.

Epoch 102: ReduceLROnPlateau reducing learning rate to 7.812500371073838e-06.

Epoch 112: ReduceLROnPlateau reducing learning rate to 3.906250185536919e-06.

Epoch 122: ReduceLROnPlateau reducing learning rate to 1.9531250927684596e-06.

Epoch 132: ReduceLROnPlateau reducing learning rate to 9.765625463842298e-07.

Epoch 142: ReduceLROnPlateau reducing learning rate to 4.882812731921149e-07.

Epoch 152: ReduceLROnPlateau reducing learning rate to 2.4414063659605745e-07.

Epoch 162: ReduceLROnPlateau reducing learning rate to 

<IPython.core.display.Javascript object>